# Bim_Predict NoteBook

## Importing Libraries

In [1]:
# Import libraries
import os
import pandas as pd

# Define project folder paths
# Data directories
BASE_DIR = "../../"
DATA_DIR = os.path.join(BASE_DIR, "data")
RAW_DATA_DIR = os.path.join(DATA_DIR, "raw_data")
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, "processed_data")
PREDICTED_DATA_DIR = os.path.join(DATA_DIR, "predicting_data")
TESTING_DATA_DIR = os.path.join(DATA_DIR, "testing_data")

# Model directories
MODELS_DIR = os.path.join(BASE_DIR, "models")
ML_MODELS_DIR = os.path.join(MODELS_DIR, "SK/machine_learning")
DL_MODELS_DIR = os.path.join(MODELS_DIR, "SK/deep_learning")
OTHER_MODELS_DIR = os.path.join(MODELS_DIR, "SK/other")

# Python modules and plots directories
PYTHON_MODULES_DIR = os.path.join(BASE_DIR, "python_modules")
PLOTS_DIR = os.path.join(BASE_DIR, "plots")

# List of directories to create
directories = [
    RAW_DATA_DIR, PROCESSED_DATA_DIR, PREDICTED_DATA_DIR,
    MODELS_DIR, ML_MODELS_DIR, DL_MODELS_DIR, OTHER_MODELS_DIR,
    PYTHON_MODULES_DIR, PLOTS_DIR
]

# Create directories if they don't exist
for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created directory: {directory}")
    else:
        print(f"Directory already exists: {directory}")

Directory already exists: ../../data/raw_data
Directory already exists: ../../data/processed_data
Directory already exists: ../../data/predicting_data
Directory already exists: ../../models
Directory already exists: ../../models/SK/machine_learning
Directory already exists: ../../models/SK/deep_learning
Directory already exists: ../../models/SK/other
Directory already exists: ../../python_modules
Directory already exists: ../../plots


<!-- ### Paths Creating && Data Importing -->

In [2]:
import os
import pandas as pd

# List all Excel files in RAW_DATA_DIR
excel_files = [f for f in os.listdir(RAW_DATA_DIR) if f.endswith(".xlsx") or f.endswith(".xls")]

# Dictionary to store DataFrames for each file and sheet
dataframes = {}

# Process each Excel file
for file in excel_files:
    file_path = os.path.join(RAW_DATA_DIR, file)
    print(f"Loading: {file_path}")

    try:
        # Load Excel file
        excel_data = pd.ExcelFile(file_path)

        # Load all sheets dynamically
        for sheet_name in excel_data.sheet_names:
            df = excel_data.parse(sheet_name)

            # Save DataFrame with a unique identifier
            dataframes[f"{file}_{sheet_name}"] = df

    except Exception as e:
        print(f"Error loading {file_path}: {e}")

# Display summary of loaded data
print(f"\nTotal files processed: {len(dataframes)}")
for key, df in dataframes.items():
    print(f"Loaded DataFrame: {key}, Shape: {df.shape}")

Loading: ../../data/raw_data/maquette_23017.xlsx
Loading: ../../data/raw_data/maquette_23016.xlsx
Loading: ../../data/raw_data/maquette_23002.xlsx
Loading: ../../data/raw_data/maquette_23007.xlsx
Loading: ../../data/raw_data/RawData-Cibles.xlsx
Loading: ../../data/raw_data/maquette_23001.xlsx

Total files processed: 23
Loaded DataFrame: maquette_23017.xlsx_Murs, Shape: (215, 149)
Loaded DataFrame: maquette_23017.xlsx_Sols, Shape: (29, 140)
Loaded DataFrame: maquette_23017.xlsx_Poutres, Shape: (152, 136)
Loaded DataFrame: maquette_23017.xlsx_Poteaux, Shape: (72, 111)
Loaded DataFrame: maquette_23016.xlsx_Murs, Shape: (1589, 146)
Loaded DataFrame: maquette_23016.xlsx_Sols, Shape: (45, 142)
Loaded DataFrame: maquette_23016.xlsx_Poutres, Shape: (778, 136)
Loaded DataFrame: maquette_23016.xlsx_Poteaux, Shape: (215, 110)
Loaded DataFrame: maquette_23002.xlsx_Murs, Shape: (345, 94)
Loaded DataFrame: maquette_23002.xlsx_Sols, Shape: (32, 91)
Loaded DataFrame: maquette_23002.xlsx_Poutres, Shape

<!-- ### Data Cleaning && PreProcessing -->

## PreProcessing Data

In [3]:
# # Define required columns dynamically
# required_columns = {
#     "Murs": ["Id", "011EC_Lot", "012EC_Ouvrage", "013EC_Localisation", "014EC_Mode Constructif", "Hauteur",
#              "Epaisseur", "AI", "AS", "Sols en intersection", "Sols coupés (u)", "Sols coupés (Ids)",
#              "Sols coupants (u)", "Sols coupants (Ids)", "Sol au-dessus", "Sol au-dessous", "Fenêtres", "Portes",
#              "Ouvertures", "Murs imbriqués", "Mur multicouche", "Mur empilé", "Profil modifié", "Extension inférieure",
#              "Extension supérieure", "Partie inférieure attachée", "Partie supérieure attachée", "Décalage supérieur",
#              "Décalage inférieur", "Matériau structurel"],

#     "Sols": ["Id", "011EC_Lot", "012EC_Ouvrage", "013EC_Localisation", "014EC_Mode Constructif", "Murs en intersection",
#              "Murs coupés (u)", "Murs coupés (Ids)", "Murs coupants (u)", "Murs coupants (Ids)", "Poutres en intersection",
#              "Poutres coupés (u)", "Poutres coupés (Ids)", "Poutres coupants (u)", "Poutres coupants (Ids)",
#              "Poteaux en intersection", "Poteaux coupés (u)", "Poteaux coupés (Ids)", "Poteaux coupants (u)",
#              "Poteaux coupants (Ids)", "Ouvertures", "Sol multicouche", "Profil modifié", "Décalage par rapport au niveau",
#              "Epaisseur", "Lié au volume", "Etude de l'élévation à la base", "Etude de l'élévation en haut",
#              "Epaisseur du porteur", "Elévation au niveau du noyau inférieur", "Elévation au niveau du noyau supérieur",
#              "Elévation en haut", "Elévation à la base", "Matériau structurel"],

#     "Poutres": ["Id", "011EC_Lot", "012EC_Ouvrage", "013EC_Localisation", "014EC_Mode Constructif", "AI", "AS",
#                 "Hauteur totale", "Hauteur", "Sols en intersection", "Sols coupés (u)", "Sols coupés (Ids)",
#                 "Sols coupants (u)", "Sols coupants (Ids)", "Sol au-dessus", "Sol au-dessous", "Poteaux en intersection",
#                 "Poteaux coupés (u)", "Poteaux coupés (Ids)", "Poteaux coupants (u)", "Poteaux coupants (Ids)",
#                 "Etat de la jonction", "Valeur de décalage Z", "Justification Z", "Valeur de décalage Y", "Justification Y",
#                 "Justification YZ", "Matériau structurel", "Elévation du niveau de référence", "Elévation en haut",
#                 "Rotation de la section", "Orientation", "Décalage du niveau d'arrivée", "Décalage du niveau de départ",
#                 "Elévation à la base", "Longueur de coupe", "Longueur", "hauteur_section", "largeur_section"],

#     "Poteaux": ["Id", "011EC_Lot", "012EC_Ouvrage", "013EC_Localisation", "014EC_Mode Constructif", "AI", "AS",
#                 "Hauteur", "Longueur", "Partie inférieure attachée", "Partie supérieure attachée", "Sols en intersection",
#                 "Sols coupés (u)", "Sols coupés (Ids)", "Sols coupants (u)", "Sols coupants (Ids)", "Poutres en intersection",
#                 "Poutres coupés (u)", "Poutres coupés (Ids)", "Poutres coupants (u)", "Poutres coupants (Ids)",
#                 "Matériau structurel", "Décalage supérieur", "Décalage inférieur", "Diamètre poteau", "h", "b",
#                 "hauteur_section", "largeur_section"]
# }

# # Initialize a dictionary to store filtered dataframes
# cleaned_dataframes = {}

# for df_name, df in dataframes.items():
#     print(f"\n🟢 Original shape of {df_name}: {df.shape}")

#     # Automatically detect the correct category for filtering
#     for category, columns in required_columns.items():
#         if category.lower() in df_name.lower():  # Match dynamically
#             try:
#                 filtered_df = df[columns]  # Keep only the required columns
#             except KeyError as e:
#                 missing_columns = set(columns) - set(df.columns)
#                 print(f"⚠️ Missing columns in {df_name}: {missing_columns}. Skipping this dataframe.")
#                 continue
#             cleaned_dataframes[df_name] = filtered_df
#             print(f"✅ Shape after filtering {df_name}: {filtered_df.shape}")
#             break  # Stop looping once the correct match is found
#     else:
#         print(f"⚠️ No matching category for {df_name}, skipping filtering.")

# # Add prefixes to column names based on the dataframe category and update index
# for name, df in cleaned_dataframes.items():
#     if "murs" in name.lower():
#         prefix = "murs_"
#     elif "sols" in name.lower():
#         prefix = "sols_"
#     elif "poutres" in name.lower():
#         prefix = "poutres_"
#     elif "poteaux" in name.lower():
#         prefix = "poteaux_"
#     else:
#         prefix = ""

#     # Rename columns with the prefix
#     df.rename(columns=lambda col: f"{prefix}{col}" if col.lower() != "id" else f"{prefix}id", inplace=True)

#     # Drop the existing index and set the prefixed ID column as the new index
#     id_column = f"{prefix}id"
#     if id_column in df.columns:
#         df.set_index(id_column, inplace=True)
#         print(f"✅ Set '{id_column}' as index for {name}.")
#     else:
#         print(f"⚠️ '{id_column}' column not found in {name}, skipping index setting.")

    # Update the cleaned_dataframes dictionary
    # cleaned_dataframes[df_name] = df

In [4]:
# print("\n📊 Cleaned DataFrames:")
# for df_name, df in cleaned_dataframes.items():
#     print(f" - {df_name}: {df.shape}")


In [5]:
# # Remove duplicates and 100% missing columns (with exceptions)
# def process_dataframe(df_name, exception_keywords):
#     # Access the dataframe from cleaned_dataframes
#     df = cleaned_dataframes[df_name].copy()

#     # Remove duplicate rows
#     initial_shape = df.shape
#     df.drop_duplicates(inplace=True)
#     duplicates_removed = initial_shape[0] - df.shape[0]
#     if duplicates_removed > 0:
#         print(f"🟢 Removed {duplicates_removed} duplicate rows from {df_name}.")
#     else:
#         print(f"✅ No duplicate rows found in {df_name}.")

#     # Identify fully missing columns
#     missing_cols = df.columns[df.isnull().mean() == 1]
#     # Keep columns that contain an exception keyword or are in target_columns intact
#     cols_to_drop = [
#         col for col in missing_cols
#         if not any(keyword in col.lower() for keyword in exception_keywords)
#     ]

#     if cols_to_drop:
#         print(f"🟠 Dropping fully missing columns from {df_name}: {cols_to_drop}")
#         df.drop(columns=cols_to_drop, inplace=True)
#     else:
#         print(f"✅ No columns dropped from {df_name}; all fully missing columns are exceptions.")

#     # Fill missing values in columns containing "coupé" or "coupants"
#     columns_to_fill = [col for col in df.columns if "coupé" in col.lower() or "coupants" in col.lower()]
#     if columns_to_fill:
#         print(f"🔵 Filling missing values with 0 for columns in {df_name}: {columns_to_fill}")
#         df[columns_to_fill] = df[columns_to_fill].fillna(0)

#     return df

# # Apply processing on all dataframes in cleaned_dataframes
# exception_keywords = ["coupés", "coupants"]

# processed_dataframes = {}
# for df_name in cleaned_dataframes.keys():
#     print(f"\n🔍 Processing dataframe: {df_name}")
#     processed_dataframes[df_name] = process_dataframe(df_name, exception_keywords)
#     final_shape = processed_dataframes[df_name].shape
#     print(f"✅ Final shape of {df_name}: {final_shape}")

In [6]:
# # Identify target columns dynamically across all DataFrames
# TARGET_COLUMNS = ['011EC_Lot', '012EC_Ouvrage', '013EC_Localisation', '014EC_Mode Constructif']

# # Check and add missing target columns
# for df_name, df in processed_dataframes.items():
#     print(f"\nProcessing dataframe: {df_name}")
#     initial_shape = df.shape  # Store the initial shape of the dataframe

#     for target in TARGET_COLUMNS:
#         if target in df.columns:
#             print(f"✅ Target column '{target}' found in dataframe '{df_name}'.")

#             # Check for missing data in the target column
#             missing_count = df[target].isnull().sum()
#             total_count = len(df)
#             missing_percentage = (missing_count / total_count) * 100
#             if missing_count > 0:
#                 print(f"⚠️ Target column '{target}' has {missing_count} missing values ({missing_percentage:.2f}%).")

#                 # Drop rows if missing data is less than 10%
#                 if missing_percentage < 10:
#                     df = df[df[target].notnull()]
#                     print(f"✅ Dropped rows with missing values in '{target}' (less than 10%).")
#             else:
#                 print(f"✅ Target column '{target}' has no missing values.")
#         else:
#             print(f"⚠️ Target column '{target}' does not exist in dataframe '{df_name}'. Adding it...")
#             # Add the missing target column with default values (e.g., NaN)
#             df[target] = float('nan')
#             print(f"✅ Added missing target column '{target}' to dataframe '{df_name}'.")

#     final_shape = df.shape  # Store the final shape of the dataframe
#     if initial_shape != final_shape:
#         print(f"📊 Shape before: {initial_shape}, Shape after: {final_shape}")

#     # Update the cleaned_dataframes dictionary
#     cleaned_dataframes[df_name] = processed_dataframes[df_name]


<!-- ### Exploratory Data Analysis (EDA) -->

In [7]:
# # Ensure missing values are filled in the processed datasets unless in TARGET_COLUMNS
# for df_name, df in cleaned_dataframes.items():
#     print(f"\n🟢 Filling missing values for {df_name}...")

#     # Display shape before filling missing values
#     initial_shape = df.shape
#     print(f"📌 Initial shape before filling NaN: {initial_shape}")

#     # Fill missing values with 0 for non-target columns
#     non_target_columns = [col for col in df.columns if col not in TARGET_COLUMNS]
#     df[non_target_columns] = df[non_target_columns].fillna(0)

#     # Store updated dataframe back
#     cleaned_dataframes[df_name] = df

#     # Display shape after processing
#     final_shape = df.shape
#     print(f"✅ Final shape after filling NaN: {final_shape}")

# print("🚀 Missing values successfully handled across all datasets!")

## EDA - Exploratory Data Analysis

In [8]:
# import pandas as pd
# import numpy as np
# from sklearn.feature_selection import VarianceThreshold
# from sklearn.preprocessing import StandardScaler

# # Function to remove low-variance & highly correlated features
# def optimize_feature_selection(df, variance_threshold=0.02, correlation_threshold=0.98):
#     print(f"\n🔍 Processing {df.shape[0]} rows & {df.shape[1]} columns")

#     # Step 1: Remove Low-Variance Features
#     selector = VarianceThreshold(variance_threshold)
#     numeric_df = df.select_dtypes(include=["number"])  # Focus only on numerical columns
#     selector.fit(numeric_df)

#     low_variance_cols = numeric_df.columns[~selector.get_support()]
#     keep_cols = [col for col in low_variance_cols if any(keyword in col.lower() for keyword in ["coupés", "coupants"])]
#     drop_cols = [col for col in low_variance_cols if col not in keep_cols and col not in TARGET_COLUMNS]

#     df.drop(columns=drop_cols, inplace=True)
#     print(f"⚠️ Dropped {len(drop_cols)} low-variance columns (excluding 'coupés' and target columns): {drop_cols}")

#     # Step 2: Remove Highly Correlated Features
#     numeric_df = df.select_dtypes(include=["number"])
#     correlation_matrix = numeric_df.corr().abs()
#     upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
#     correlated_features = [
#         col for col in upper_triangle.columns
#         if any(upper_triangle[col] > correlation_threshold) and col not in TARGET_COLUMNS
#     ]

#     df.drop(columns=correlated_features, inplace=True)
#     print(f"⚠️ Dropped {len(correlated_features)} highly correlated columns (excluding target columns): {correlated_features}")

#     print(f"✅ Final shape after filtering: {df.shape}")
#     return df

# # Apply optimized feature selection to all datasets
# final_cleaned_dataframes = {name: optimize_feature_selection(df) for name, df in cleaned_dataframes.items()}

# print("🚀 Optimized feature selection completed successfully!")

In [9]:
# # Display basic statistics for all cleaned sheets
# for df_name, df in final_cleaned_dataframes.items():
#     print(f"\nSummary statistics for {df_name}:")

#     print(df.describe())

<!-- ### Feature Selection -->

In [10]:
# # Identify target columns dynamically across all DataFrames
# target_columns_found = set()
# for df_name, df in final_cleaned_dataframes.items():
#     found_targets = [
#         col for col in df.columns
#         if any(target.lower() in col.lower() for target in TARGET_COLUMNS)
#     ]
#     target_columns_found.update(found_targets)

# print(f"\nTarget columns detected across datasets: {target_columns_found}")

<!-- ## Training and testing  -->

## Deep-Learning Section

In [11]:
import os
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection  import train_test_split
from sklearn.preprocessing    import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.impute           import SimpleImputer
from sklearn.compose          import ColumnTransformer
from sklearn.pipeline         import Pipeline

import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers


2025-06-10 13:37:55.784140: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-10 13:37:56.087159: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-06-10 13:37:56.087198: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-06-10 13:37:56.147514: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-10 13:37:58.166644: W tensorflow/stream_executor/platform/de

In [12]:
# ─── 1. DATA CLEANING ─────────────────────────────────────────────────────────
def filter_required_columns(dataframes, required_columns):
    """
    For each df in `dataframes`, pick the matching sheet key and
    keep only its required columns. Returns a dict of filtered dfs.
    """
    cleaned = {}
    for name, df in dataframes.items():
        for sheet, cols in required_columns.items():
            if sheet.lower() in name.lower():
                missing = set(cols) - set(df.columns)
                if missing:
                    print(f"⚠️ Missing in {name}: {missing}. Skipping.")
                else:
                    cleaned[name] = df[cols].copy()
                    print(f"✅ {name}: filtered to {df[cols].shape}")
                break
        else:
            print(f"⚠️ No match for {name}, skipped")
    return cleaned


def drop_duplicates_and_missing(df, exception_keywords):
    """
    - Drop duplicate rows
    - Drop any column that’s 100% NaN unless it contains an exception keyword
    - Fill any “coupé”/“coupants” column’s NaNs with 0
    """
    before = df.shape[0]
    df = df.drop_duplicates()
    print(f"🗑️  Dups removed: {before - df.shape[0]}")

    fully_missing = [c for c in df.columns if df[c].isna().all()]
    to_drop = [c for c in fully_missing
               if not any(kw.lower() in c.lower() for kw in exception_keywords)]
    df = df.drop(columns=to_drop)
    if to_drop: print(f"🗑️  Dropped 100% NaN cols: {to_drop}")

    fill_cols = [c for c in df.columns if "coupé" in c.lower() or "coupant" in c.lower()]
    df[fill_cols] = df[fill_cols].fillna(0)
    if fill_cols: print(f"ℹ️  Filled coupé cols with 0: {fill_cols}")

    return df


def ensure_targets(df, target_columns, drop_threshold=0.10):
    """
    - Ensure each target exists (if not, create it with NaN)
    - If existing but < drop_threshold% missing → drop those rows
    - Leave > threshold as-is (you may choose to impute later)
    """
    for t in target_columns:
        if t not in df.columns:
            df[t] = np.nan
            print(f"➕ Added missing target {t}")
        miss_pct = df[t].isna().mean()
        if 0 < miss_pct < drop_threshold:
            df = df[df[t].notna()]
            print(f"🗑️ Dropped rows where {t} was NaN ({miss_pct:.1%})")
        else:
            print(f"ℹ️  {t}: {miss_pct:.1%} missing")
    return df


def fill_non_targets(df, target_columns):
    """
    Fill all non‐target columns’ NaNs with 0
    """
    non_targets = [c for c in df.columns if c not in target_columns]
    df[non_targets] = df[non_targets].fillna(0)
    print(f"✅ Filled NaN→0 on non‐targets ({len(non_targets)} cols)")
    return df


def clean_all_dataframes(
    raw_dfs,
    required_columns,
    exception_keywords,
    target_columns
):
    # 1) filter
    filt = filter_required_columns(raw_dfs, required_columns)

    # 2) per‐df cleaning
    cleaned = {}
    for name, df in filt.items():
        print(f"\n🔍 Cleaning {name}")
        df2 = drop_duplicates_and_missing(df, exception_keywords)
        df3 = ensure_targets(df2, target_columns)
        df4 = fill_non_targets(df3,   target_columns)
        cleaned[name] = df4
        print(f"🎯 Final {name} shape: {df4.shape}")
    return cleaned


In [13]:
# ─── 2. PREPROCESS & SPLIT ────────────────────────────────────────────────────
def build_preprocessor(X_df):
    """
    - numeric: mean‐impute + StandardScaler
    - categorical: mode‐impute + OneHotEncoder (ensuring all values are strings)
    """
    num_cols = X_df.select_dtypes(["int64", "float64"]).columns
    cat_cols = X_df.select_dtypes(["object", "category"]).columns

    # Convert categorical columns to string to avoid mixed types
    X_df[cat_cols] = X_df[cat_cols].astype(str)

    num_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler()),
    ])
    cat_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(sparse=False, handle_unknown="ignore")),
    ])
    transformer = ColumnTransformer([
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols),
    ])
    return transformer


def prepare_dataset(
    df,
    target_columns,
    test_size=0.3,
    val_size=0.5,
    random_state=42
):
    """
    - Splits df into X, y
    - LabelEncodes each target
    - Preprocesses X
    - Splits into train/val/test
    """
    X = df.drop(columns=target_columns)
    y = df[target_columns].astype(str)
    # encode Y
    le_dict, y_enc = {}, []
    for col in y:
        le = LabelEncoder().fit(y[col])
        y_enc.append(le.transform(y[col]))
        le_dict[col] = le
    y_enc = np.vstack(y_enc).T

    # preprocess X
    pre = build_preprocessor(X)
    X_all = pre.fit_transform(X)

    # splits
    strat = y_enc[:,0]
    X_tr, X_tmp, y_tr, y_tmp = train_test_split(
        X_all, y_enc, test_size=test_size,
        stratify=strat, random_state=random_state
    )
    X_val, X_te, y_val, y_te = train_test_split(
        X_tmp, y_tmp, test_size=val_size,
        stratify=y_tmp[:,0], random_state=random_state
    )
    return (X_tr,y_tr), (X_val,y_val), (X_te,y_te), pre, le_dict


In [14]:
# ─── 3. MODEL BUILDING & TRAINING ────────────────────────────────────────────

def build_mlp(input_dim, output_dims, hp):
    inp = layers.Input((input_dim,))
    x = inp
    for i in range(hp["num_layers"]):
        x = layers.Dense(hp["units"], activation="relu")(x)
        if hp["dropout"]>0:
            x = layers.Dropout(hp["dropout"])(x)

    outputs, losses, metrics = [], {}, {}
    for i, d in enumerate(output_dims):
        name = f"out{i}"
        o = layers.Dense(d, activation="softmax", name=name)(x)
        outputs.append(o)
        losses[name]   = "sparse_categorical_crossentropy"
        metrics[name]  = ["accuracy"]

    m = Model(inp, outputs)
    m.compile(
        optimizer=optimizers.Adam(hp["lr"]),
        loss=losses,
        metrics=metrics
    )
    return m


def train_models(
    X_tr, y_tr,
    X_val, y_val,
    output_dims,
    hyperparams,
    epochs=30,
    batch_size=32
):
    y_tr_dict  = {f"out{i}": y_tr[:,i] for i in range(y_tr.shape[1])}
    y_val_dict = {f"out{i}": y_val[:,i] for i in range(y_val.shape[1])}

    models, histories, scores = [], [], []
    for hp in hyperparams:
        tf.keras.backend.clear_session()
        m = build_mlp(X_tr.shape[1], output_dims, hp)
        h = m.fit(
            X_tr, y_tr_dict,
            validation_data=(X_val, y_val_dict),
            epochs=epochs, batch_size=batch_size, verbose=0
        )
        ev = m.evaluate(X_val, y_val_dict, verbose=0)
        models.append(m)
        histories.append((hp, h))
        scores.append((hp, ev))
        print(f"🏷 hp={hp} → val_loss={ev[0]:.4f}")
    return models, histories, scores


def plot_learning_curves(histories, title):
    plt.figure(figsize=(6,4))
    for hp,h in histories:
        lbl = f"{hp['units']}u×{hp['num_layers']}L dr={hp['dropout']} lr={hp['lr']}"
        plt.plot(h.history["val_loss"], label=lbl)
    plt.title(title); plt.xlabel("epoch"); plt.ylabel("val_loss")
    plt.legend(); plt.tight_layout(); plt.show()


def summarize_scores(scores):
    rows = []
    for hp, ev in scores:
        row = {**hp, "total_val_loss": ev[0]}
        idx = 1
        head = 0
        while idx<len(ev):
            row[f"o{head}_loss"] = ev[idx]
            row[f"o{head}_acc"]  = ev[idx+1]
            idx+=2; head+=1
        rows.append(row)
    return pd.DataFrame(rows).sort_values("total_val_loss").reset_index(drop=True)


def save_top3(models, scores, prefix):
    losses = np.array([s[1][0] for s in scores])
    best   = np.argsort(losses)[:3]
    for rank,i in enumerate(best,1):
        fn = f"{prefix}_top{rank}.h5"
        models[i].save(fn)
        print(f"💾 Saved {fn}")


def test_saved_models(
    model_paths, preprocessor, label_encoders,
    seen_excel_paths, sheet_key,
    required_columns, target_columns
):
    for mp in model_paths:
        print(f"\n🔎 Testing {mp}")
        m = tf.keras.models.load_model(mp)
        for xp in seen_excel_paths:
            df = pd.read_excel(xp, sheet_name=sheet_key)
            df = df[required_columns[sheet_key]]
            Xn = preprocessor.transform(df.drop(columns=target_columns))
            preds = m.predict(Xn)
            out = df.copy()
            for i,col in enumerate(target_columns):
                le = label_encoders[col]
                out[f"pred_{col}"] = le.inverse_transform(preds[i].argmax(axis=1))
            print(f"— on {os.path.basename(xp)}:")
            print(out.head())
            print()


In [20]:
# ─── 4. MAIN EXECUTION ────────────────────────────────────────────────────────

if __name__ == "__main__":
    # 1) load your raw Excel sheets however you like:
    # Load all Excel files in RAW_DATA_DIR
    raw_dfs = {}
    for file in excel_files:
        file_path = os.path.join(RAW_DATA_DIR, file)
        print(f"Loading: {file_path}")
        try:
            excel_data = pd.ExcelFile(file_path)
            for sheet_name in excel_data.sheet_names:
                df_name = f"{file}_{sheet_name}"
                raw_dfs[df_name] = excel_data.parse(sheet_name)
                print(f"Loaded sheet: {sheet_name} from {file}, Shape: {raw_dfs[df_name].shape}")
        except Exception as e:
            print(f"Error loading {file_path}: {e}")

    # 2) your parameters
    EXC_KEYWORDS      = ["coupés", "coupants"]
    TARGET_COLS       = ["011EC_Lot", "012EC_Ouvrage", "013EC_Localisation", "014EC_Mode Constructif"]
    SEEN_FILES        = ["seen1.xlsx", "seen2.xlsx"]
    HYPERPARAM_GRID   = [
        {"num_layers": 1, "units": 64, "dropout": 0.0, "lr": 1e-3},
        {"num_layers": 2, "units": 64, "dropout": 0.2, "lr": 1e-3},
        {"num_layers": 2, "units": 128, "dropout": 0.3, "lr": 5e-4},
        {"num_layers": 3, "units": 256, "dropout": 0.4, "lr": 1e-4},
    ]

    # 3) CLEAN
    cleaned = {}
    for name, df in raw_dfs.items():
        print(f"\n🔍 Cleaning {name}")
        df2 = drop_duplicates_and_missing(df, EXC_KEYWORDS)
        df3 = ensure_targets(df2, TARGET_COLS)
        df4 = fill_non_targets(df3, TARGET_COLS)
        cleaned[name] = df4
        print(f"🎯 Final {name} shape: {df4.shape}")

    # 4) FOR EACH SHEET → preprocess, train, eval, save, test
    for name, df in cleaned.items():
        print(f"\n\n===== PROCESSING {name} =====")

        # Ensure all classes in the target columns have at least two samples
        for target in TARGET_COLS:
            class_counts = df[target].value_counts()
            valid_classes = class_counts[class_counts >= 2].index
            if len(valid_classes) < 2:
                print(f"⚠️ Skipping {name} due to insufficient valid classes in {target}.")
                df = pd.DataFrame()  # Clear the dataframe to avoid further processing
                break
            df = df[df[target].isin(valid_classes)]
            print(f"Filtered {target}: Removed classes with fewer than 2 samples.")

        # preprocess & split
        (X_tr, y_tr), (X_val, y_val), (X_te, y_te), preproc, le_dict = prepare_dataset(
            df, TARGET_COLS, test_size=0.3, val_size=0.5, random_state=42
        )
        out_dims = [len(le_dict[c].classes_) for c in TARGET_COLS]

        # train
        models, hists, scores = train_models(
            X_tr, y_tr, X_val, y_val,
            output_dims=out_dims,
            hyperparams=HYPERPARAM_GRID,
            epochs=40
        )

        # visualize & table
        plot_learning_curves(hists, title=f"{name} val_loss")
        df_scores = summarize_scores(scores)
        print(df_scores)


Loading: ../../data/raw_data/maquette_23017.xlsx
Loaded sheet: Murs from maquette_23017.xlsx, Shape: (215, 149)
Loaded sheet: Sols from maquette_23017.xlsx, Shape: (29, 140)
Loaded sheet: Poutres from maquette_23017.xlsx, Shape: (152, 136)
Loaded sheet: Poteaux from maquette_23017.xlsx, Shape: (72, 111)
Loading: ../../data/raw_data/maquette_23016.xlsx
Loaded sheet: Murs from maquette_23016.xlsx, Shape: (1589, 146)
Loaded sheet: Sols from maquette_23016.xlsx, Shape: (45, 142)
Loaded sheet: Poutres from maquette_23016.xlsx, Shape: (778, 136)
Loaded sheet: Poteaux from maquette_23016.xlsx, Shape: (215, 110)
Loading: ../../data/raw_data/maquette_23002.xlsx
Loaded sheet: Murs from maquette_23002.xlsx, Shape: (345, 94)
Loaded sheet: Sols from maquette_23002.xlsx, Shape: (32, 91)
Loaded sheet: Poutres from maquette_23002.xlsx, Shape: (96, 89)
Loading: ../../data/raw_data/maquette_23007.xlsx
Loaded sheet: Murs from maquette_23007.xlsx, Shape: (203, 91)
Loaded sheet: Sols from maquette_23007.xl

/tmp/ipykernel_38590/4147864326.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[non_targets] = df[non_targets].fillna(0)


🗑️  Dropped 100% NaN cols: ["Type prédéfini d'IFC", 'Exporter au format IFC sous', 'Commentaires', 'Niveau', 'Nom de la famille', 'Nom du type', '001EC_Grue', '003EC_Zone', '021EC_Specificite', '041EC_Option Logetex 1', '042EC_Option Logetex 2', '043EC_Option Logetex 3', '051EC_Date Realisation', '054EC_Jour Coulage', '055EC_Jour Coulage Cumule', '084EC_Clef Planification', '085EC_Clef Grue', '024EC_Finition GO', '056EC_Heure Grue', '087EC_Clef AB', '088EC_Clef BMO', "Note d'identification", "Type: Type prédéfini d'IFC", 'Exporter le type au format IFC sous', 'Modèle', 'Fabricant', 'Commentaires du type', 'URL', 'Description', 'Nom de code', 'Identifiant du nom de la coupe', 'Titre OmniClass', 'Numéro OmniClass', "Description de l'assemblage", "Code d'assemblage", 'Marque de type', "Protection contre l'incendie"]
ℹ️  Filled coupé cols with 0: ['Sols coupés (u)', 'Sols coupés (Ids)', 'Sols coupants (u)', 'Sols coupants (Ids)']
ℹ️  011EC_Lot: 0.0% missing
ℹ️  012EC_Ouvrage: 0.0% missing


/tmp/ipykernel_38590/4147864326.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[non_targets] = df[non_targets].fillna(0)
/tmp/ipykernel_38590/4147864326.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[non_targets] = df[non_targets].fillna(0)


✅ Filled NaN→0 on non‐targets (61 cols)
🎯 Final maquette_23007.xlsx_Sols shape: (37, 65)

🔍 Cleaning maquette_23007.xlsx_Poutres
🗑️  Dups removed: 0
🗑️  Dropped 100% NaN cols: ["Type prédéfini d'IFC", 'Exporter au format IFC sous', 'Niveau', 'Nom de la famille', 'Nom du type', 'Identifiant', 'NOEMI', 'MILLS_View', "Note d'identification", "Type: Type prédéfini d'IFC", 'Exporter le type au format IFC sous', 'Modèle', 'Fabricant', 'Commentaires du type', 'URL', 'Description', 'Nom de code', 'Identifiant du nom de la coupe', "Description de l'assemblage", "Code d'assemblage", 'Marque de type', "Protection contre l'incendie"]
ℹ️  Filled coupé cols with 0: ['Sols coupés (u)', 'Sols coupés (Ids)', 'Sols coupants (u)', 'Sols coupants (Ids)', 'Poteaux coupés (u)', 'Poteauc coupés (Ids)', 'Poteaux coupants (u)', 'Poteaux coupants (Ids)']
🗑️ Dropped rows where 011EC_Lot was NaN (4.5%)
ℹ️  012EC_Ouvrage: 0.0% missing
ℹ️  013EC_Localisation: 0.0% missing
ℹ️  014EC_Mode Constructif: 0.0% missing
✅ 

/tmp/ipykernel_38590/4147864326.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[non_targets] = df[non_targets].fillna(0)


✅ Filled NaN→0 on non‐targets (63 cols)
🎯 Final RawData-Cibles.xlsx_Poteaux shape: (68, 67)

🔍 Cleaning maquette_23001.xlsx_Murs
🗑️  Dups removed: 0
🗑️  Dropped 100% NaN cols: ["Type prédéfini d'IFC", 'Exporter au format IFC sous', 'Commentaires', 'Nom de la famille', 'Nom du type', 'Batiment', "Note d'identification", "Type: Type prédéfini d'IFC", 'Exporter le type au format IFC sous', 'Fabricant', 'Commentaires du type', 'URL', 'Description', "Description de l'assemblage", "Code d'assemblage", 'Motif vue détail faible', 'Marque de type', "Protection contre l'incendie"]
ℹ️  Filled coupé cols with 0: ['Sols coupés (u)', 'Sols coupés (Ids)', 'Sols coupants (u)', 'Sols coupants (Ids)']
ℹ️  011EC_Lot: 0.0% missing
ℹ️  012EC_Ouvrage: 0.0% missing
ℹ️  013EC_Localisation: 0.0% missing
ℹ️  014EC_Mode Constructif: 0.0% missing
✅ Filled NaN→0 on non‐targets (74 cols)
🎯 Final maquette_23001.xlsx_Murs shape: (312, 78)

🔍 Cleaning maquette_23001.xlsx_Sols
🗑️  Dups removed: 0
🗑️  Dropped 100% NaN c

/tmp/ipykernel_38590/4147864326.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[non_targets] = df[non_targets].fillna(0)


✅ Filled NaN→0 on non‐targets (76 cols)
🎯 Final maquette_23001.xlsx_Poutres shape: (242, 80)

🔍 Cleaning maquette_23001.xlsx_Poteaux
🗑️  Dups removed: 0
🗑️  Dropped 100% NaN cols: ["Type prédéfini d'IFC", 'Exporter au format IFC sous', 'Commentaires', 'Nom de la famille', 'Nom du type', 'Batiment', "Note d'identification", "Type: Type prédéfini d'IFC", 'Exporter le type au format IFC sous', 'Modèle', 'Fabricant', 'Commentaires du type', 'URL', 'Description', 'Nom de code', 'Identifiant du nom de la coupe', 'Titre OmniClass', 'Numéro OmniClass', "Description de l'assemblage", 'Marque de type']
ℹ️  Filled coupé cols with 0: ['Sols coupés (u)', 'Sols coupés (Ids)', 'Sols coupants (u)', 'Sols coupants (Ids)', 'Poutres coupés (u)', 'Poutres coupés (Ids)', 'Poutres coupants (u)', 'Poutres coupants (Ids)']
ℹ️  011EC_Lot: 0.0% missing
ℹ️  012EC_Ouvrage: 0.0% missing
ℹ️  013EC_Localisation: 0.0% missing
ℹ️  014EC_Mode Constructif: 0.0% missing
✅ Filled NaN→0 on non‐targets (63 cols)
🎯 Final maq

KeyError: "['011EC_Lot', '012EC_Ouvrage', '013EC_Localisation', '014EC_Mode Constructif'] not found in axis"

In [ ]:
        # Save & test
        prefix = sheet.lower()
        save_top3(models, scores, os.path.join(DL_MODELS_DIR, prefix))
        best_models = [os.path.join(DL_MODELS_DIR, f"{prefix}_top{i}.h5") for i in (1, 2, 3)]


In [ ]:
        # List all Excel files in TESTING_DATA_DIR
        testing_excel_files = [f for f in os.listdir(TESTING_DATA_DIR) if f.endswith(".xlsx") or f.endswith(".xls")]

        # Test saved models and save predictions
        for model_path in best_models:
            print(f"\n🔎 Testing model: {model_path}")
            model = tf.keras.models.load_model(model_path)

            for test_file in testing_excel_files:
                test_file_path = os.path.join(TESTING_DATA_DIR, test_file)
                print(f"Processing test file: {test_file_path}")

                try:
                    # Load the Excel file
                    test_excel_data = pd.ExcelFile(test_file_path)

                    # Process each sheet in the Excel file
                    for sheet_name in test_excel_data.sheet_names:
                        if sheet_name.lower() == sheet.lower():  # Match the category dynamically
                            test_df = test_excel_data.parse(sheet_name)

                            # Filter required columns
                            test_df = test_df[required_columns[sheet]]

                            # Preprocess the test data
                            X_test = preproc.transform(test_df.drop(columns=TARGET_COLUMNS))

                            # Predict using the model
                            predictions = model.predict(X_test)

                            # Add predictions to the DataFrame
                            for i, target in enumerate(TARGET_COLUMNS):
                                test_df[f"pred_{target}"] = le_dict[target].inverse_transform(predictions[i].argmax(axis=1))

                            # Save the predictions to PREDICTED_DATA_DIR
                            predicted_file_path = os.path.join(PREDICTED_DATA_DIR, f"{test_file}_{sheet_name}_predictions.xlsx")
                            test_df.to_excel(predicted_file_path, index=False)
                            print(f"✅ Predictions saved to: {predicted_file_path}")

                except Exception as e:
                    print(f"⚠️ Error processing {test_file_path}: {e}")